# Final Project Starting Guide

Hello everyone, welcome to the final project! This notebook is provided to you to reiterate the rules and guidelines, and give you some starting points.

### What we provide

In this project, we will provide you with 
- This starting guide
- A working API that you can access under ASU network (i.e., on campus or with VPN)
- A starting development data that you can use to develop your agent. It contains 1,000 instances with {domain, input, expected_output}

### Your goal

In this project, you will implement an inference-time agent to solve reasoning requests, as those provided in the development data. The grading of this project will be effort-based and you will get full credit if you produce the minimum deliverables below, with subject to the rules and requirements below.

#### Minimum Deliverables

1. A working agent loop (in the form of a Github project) that the TA can run, and implements *at least three* inference-time algorithms or techniques.
2. Outputs from your agent on the released test data (see important dates). 
3. A short one-page report on how your agent works, and pointer to important techniques (referece to code blocks).

#### Rules and Requirements
1. You must only use our provided API call to access LLMs; meaning that you cannot use any other LLMs in any other way within your agent loop. Some exceptions may be made if you call certain external tools (e.g., Google search) that use some LLMs internally. Please discuss any exceptions with us to avoid penalties up to 50% of the project grade.
2. You must not hardcode a full delegation to an external tool (e.g., google_search(input_problem)). Such delegations must be automatically selected/decided by your agent. Hardcode delegations will lead to a zero.
3. You cannot use Cursor or any AI coding aids to implement the final project. You can, however, ask LLMs (or other online resources) for conceptual clarification or code examples. Your final project should not contain any blocks of code (i.e., > 3 lines) that are written by AI. Violations will lead to a zero.
4. Your agent should be able to run efficiently, with <20 LLM calls per question. Exceptions may be made when you have a complicated agent but please discuss with us. Up to 10% of the project grade may be deducted if we observe very inefficient LLM usages that do not clearly benefit the performance.
5. Your agent must run without any requests to any paid services (paid is defined by if the TA has to pay to run it, regardless of whether you actuallly pay for it or not.) Violations will lead to a zero.
6. You must submit a Github project link as your code submission. All changes must be tracked and any commits should be within 100 lines of +/- with good messages. Points will be deducted to up to 25% of the project grade if we observe "magic commits" or too few commits. 


### Suggestions
1. Start early, please.
2. You should consider how you can evaluate whether your output is good enough compared to the provided expected_outputs, and we will not release how we will actually evaluate your outputs; meaning that you have to try to predict how we will evaluate things.
3. Start with a basic implementation, and iterate based on mistakes/feedbacks.
4. Find more development data, or create your own cases to stree-test your agent. 
5. You are free to modify any provided code in this starting guide, or not using any of these code at all.

### Important dates
- **Release of final test data**: 11/25/2025
- **Deadline for submitting all deliverables**: 12/05/2025

### Extra Credit. 
The top 20 projects (ranked by performance metrics on the test data and at the TA's discretion of implementation quality) will be given extra credits. The actual credits will be between 1% to 7.5% depending on the ranking. 

In [110]:
# %% Minimal setup
# If needed (uncomment in a notebook):
# !pip install requests python-dotenv

import os, json, textwrap, re, time
import requests

API_KEY  = os.getenv("OPENAI_API_KEY", "cse476")
API_BASE = os.getenv("API_BASE", "http://10.4.58.53:41701/v1")  
MODEL    = os.getenv("MODEL_NAME", "bens_model")              

def call_model_chat_completions(prompt: str,
                                system: str = "You are a helpful assistant. Reply with only the final answer—no explanation.",
                                model: str = MODEL,
                                temperature: float = 0.0,
                                timeout: int = 60) -> dict:
    """
    Calls an OpenAI-style /v1/chat/completions endpoint and returns:
    { 'ok': bool, 'text': str or None, 'raw': dict or None, 'status': int, 'error': str or None, 'headers': dict }
    """
    url = f"{API_BASE}/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type":  "application/json",
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user",   "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": 128,
    }

    try:
        resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
        status = resp.status_code
        hdrs   = dict(resp.headers)
        if status == 200:
            data = resp.json()
            text = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            return {"ok": True, "text": text, "raw": data, "status": status, "error": None, "headers": hdrs}
        else:
            # try best-effort to surface error text
            err_text = None
            try:
                err_text = resp.json()
            except Exception:
                err_text = resp.text
            return {"ok": False, "text": None, "raw": None, "status": status, "error": str(err_text), "headers": hdrs}
    except requests.RequestException as e:
        return {"ok": False, "text": None, "raw": None, "status": -1, "error": str(e), "headers": {}}


## 1) Smoke test: direct inference

We’ll do a single request with a strict instruction to answer briefly.  
*If you see an auth error, set `OPENAI_API_KEY` and (if needed) `API_BASE`/`MODEL_NAME`.*


In [ ]:
# %% Direct call example
demo_prompt = "What is 17 + 28? Answer with just the number."
result = call_model_chat_completions(demo_prompt)
print("OK:", result["ok"], "HTTP:", result["status"])
print("MODEL SAYS:", (result["text"] or "").strip())

# Optional: Inspect rate-limit headers if your provider exposes them
for k in ["x-ratelimit-remaining-requests", "x-ratelimit-limit-requests", "x-request-id"]:
    if k in result["headers"]:
        print(f"{k}: {result['headers'][k]}")


## 2) A tiny test set (3 questions)

We’ll cover:
1. **Math reasoning** — inequality solving,
2. **Common sense** — buoyancy/ice & water,
3. **Logic** — a classic race-position puzzle.

We also tightly constrain the required answer forms to enable simple auto‑grading.


In [ ]:
# %% Define three tests: input + expected
tests = [
    {
        "id": "math_inequality",
        "type": "numeric",  # grader will prefer numeric extraction
        "prompt": "Solve for the smallest integer n such that 3n + 5 > 26. Answer with just the integer.",
        "expected": "8",    # Because 3n > 21 => n > 7, smallest integer is 8
    },
    {
        "id": "commonsense_ice",
        "type": "text",
        "prompt": (
            "You place an ice cube in a glass of water and mark the water level. "
            "After the ice melts, does the water level rise, fall, or stay the same? "
            "Answer with exactly one of: 'rise', 'fall', 'stay the same'."
        ),
        "expected": "stay the same",
    },
    {
        "id": "logic_race",
        "type": "text",
        "prompt": (
            "In a race, you pass the person in second place. What position are you now in? "
            "Answer with a single word like 'first', 'second', 'third'."
        ),
        "expected": "second",
    },
]


## 3) Minimal evaluator

We provide some example code to decide whether the agent outputs match the expected outputs, just to give you an idea of how evaluations can be done. You are free to use this code, or not.

In [ ]:
# %% Simple normalization and evaluation helpers
def normalize_text(s: str) -> str:
    s = (s or "").strip().lower()
    # Remove surrounding punctuation and extra whitespace
    s = re.sub(r"[^\w\s\-']", " ", s)
    s = re.sub(r"\s+", " ", s).strip()

    # Map common synonyms used in these tests
    synonyms = {
        "unchanged": "stay the same",
        "no change": "stay the same",
        "same": "stay the same",
        "second place": "second",
        "2nd": "second",
        "first place": "first",
        "third place": "third",
    }
    return synonyms.get(s, s)

def extract_number(s: str):
    # Returns first number occurrence as string if found, else None
    if not s:
        return None
    m = re.search(r"[-+]?\d+(\.\d+)?", s)
    return m.group(0) if m else None

def grade(expected: str, got: str, kind: str) -> bool:
    if kind == "numeric":
        exp_num = extract_number(expected)
        got_num = extract_number(got)
        return (exp_num is not None) and (got_num == exp_num)
    else:
        return normalize_text(got) == normalize_text(expected)

def evaluate_tests(tests, model=MODEL):
    rows = []
    for t in tests:
        r = call_model_chat_completions(
            t["prompt"],
            system="You are a careful solver. Reply ONLY with the final answer, nothing else.",
            model=model,
            temperature=0.0,
        )
        got = (r["text"] or "").strip()
        is_correct = grade(t["expected"], got, t["type"])
        rows.append({
            "id": t["id"],
            "expected": t["expected"],
            "got": got,
            "correct": is_correct,
            "status": r["status"],
            "error": r["error"],
        })
        # Tiny pacing to be polite to the API
        time.sleep(0.2)

    # Print a small report
    correct = sum(1 for x in rows if x["correct"])
    print(f"Score: {correct}/{len(rows)} correct")
    for x in rows:
        mark = "✅" if x["correct"] else "❌"
        print(f"{mark} {x['id']}: expected={x['expected']!r}, got={x['got']!r} (HTTP {x['status']})")
        if x["error"]:
            print("   error:", x["error"])
    return rows

results = evaluate_tests(tests)


In [ ]:
def self_evaluate(question, prediction, expected_answer, model=MODEL):
    """
    Use the model itself as a strict grader.
    Returns True if the model says the prediction matches the expected answer; else False.
    Falls back to a simple normalized string compare if the model's reply is malformed.
    """
    import re

    system = "You are a strict grader. Reply with exactly True or False. No punctuation. No explanation."
    prompt = f"""You are grading a question-answer pair.

Return exactly True if the PREDICTION would be accepted as correct for the EXPECTED_ANSWER.
Otherwise, return False.

QUESTION:
{question}

PREDICTION:
{prediction}

EXPECTED_ANSWER:
{expected_answer}

Answer with exactly: True or False
"""

    r = call_model_chat_completions(
        prompt,
        system=system,
        model=model,
        temperature=0.0,
    )

    reply = (r.get("text") or "").strip().lower()
    if reply.startswith("true"):
        return True
    if reply.startswith("false"):
        return False

    # Fallback: simple normalization-based equality
    norm = lambda s: re.sub(r"\s+", " ", (s or "").strip().lower())
    return norm(prediction) == norm(expected_answer)


In [ ]:
def self_evaluate_tests(tests, model=MODEL, grader_model=None, sleep_sec=0.2, verbose=True):
    """
    Run the tests by querying the model for each prompt, then use LLM-as-a-judge
    (self_evaluate) to determine correctness.

    Args:
        tests: list of dicts with keys: id, prompt, expected (and optionally type)
        model: model used to generate predictions
        grader_model: model used to judge correctness (defaults to `model` if None)
        sleep_sec: small delay between calls to be polite to the API
        verbose: if True, print a summary line per test

    Returns:
        rows: list of dicts with fields:
              id, expected, got, correct, status, error
    """
    import time

    judge_model = grader_model or model
    rows = []

    for t in tests:
        # 1) Get model prediction
        r = call_model_chat_completions(
            t["prompt"],
            system="You are a careful solver. Reply ONLY with the final answer, nothing else.",
            model=model,
            temperature=0.0,
        )
        got = (r.get("text") or "").strip()

        # 2) LLM-as-a-judge: strict True/False
        is_correct = self_evaluate(
            question=t["prompt"],
            prediction=got,
            expected_answer=t["expected"],
            model=judge_model,
        )

        row = {
            "id": t.get("id", "<unnamed>"),
            "expected": t["expected"],
            "got": got,
            "correct": bool(is_correct),
            "status": r.get("status"),
            "error": r.get("error"),
        }
        rows.append(row)

        if verbose:
            mark = "✅" if is_correct else "❌"
            print(f"{mark} {row['id']}: expected={row['expected']!r}, got={row['got']!r} (HTTP {row['status']})")
            if row["error"]:
                print("   error:", row["error"])

        if sleep_sec:
            time.sleep(sleep_sec)

    return rows

# Example:
results_llm_judge = self_evaluate_tests(tests, verbose=True, model=MODEL, grader_model=MODEL)


## My Agent Implementation


In [111]:
#load development data from json
import json

with open ("cse476_final_project_dev_data.json", "r", encoding="utf-8") as f:
    dev_data = json.load(f)

print("Number of dev examples:" , len(dev_data))
print("First dev item:")
print(dev_data[0])


Number of dev examples: 1000
First dev item:
{'input': 'Let $ABCD$ be a convex quadrilateral with $AB = CD = 10$ , $BC = 14$ , and $AD = 2\\sqrt{65}$ . Assume that the diagonals of $ABCD$ intersect at point $P$ , and that the sum of the areas of triangles $APB$ and $CPD$ equals the sum of the areas of triangles $BPC$ and $APD$ . Find the area of quadrilateral $ABCD$ .', 'output': '112', 'domain': 'math'}


In [112]:
#load test data + answer template
with open("cse_476_final_project_test_data.json", "r", encoding="utf-8") as f:
    test_data = json.load(f)

with open("cse_476_final_project_answers.json", "r" , encoding="utf-8") as f:
    test_answers = json.load(f)

print("Test questions:", len(test_data))
print("Template rows:", len(test_answers))

Test questions: 6208
Template rows: 6208


Inference Techniques #1 chain of thought

In [113]:
def chain_of_thought(task_text: str, system_message: str=( "You are a careful reasoner. Think step by step, then give the final answer on the last line"),
                     temp: float=0.4):
    """ single chain of thought call to bens model. Returns (raw_text, raw_response_dict)."""
    
    res = call_model_chat_completions(prompt=task_text,system=system_message,model=MODEL,temperature=temp,)\
    
    raw_text = (res.get("text") or "").strip()
    return raw_text, res


In [114]:
txt, _ = chain_of_thought("what is 13+29? Just give the number")
print((txt))

13 + 29 = 42


In [115]:
# final answer extrtactor

def get_final_answer(text:str)->str:
    """ Get short final answer from long chain-of-thought string. 
    Look for 'FINAL ANSWER' else use last non empty line"""

    if not text:
        return ""
    # if model follow final answer use that
    match = re.search(r"FINAL ANSWER[:\-]\s*(.+)", text, flags=re.IGNORECASE)
    if match:
        return match.group(1).strip()
    
    # else, grab last number in text
    nums = re.findall(r"-?\d+(?:\.\d+)?", text)
    if nums:
        return nums[-1].strip()
    
    #last non empty line
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    if not lines:
        return ""
    return lines[-1]
    

In [116]:
print(get_final_answer("some steps\nFINAL ANSWER: 42"))

42


Inference Techniques #2 : Self consistency 

In [118]:
from collections import Counter

# self consistency

def self_con( question_text: str, 
              sample_count: int = 5,
              temp: float = 0.7,
              ):
    
    """self-consistency: run CoT multiple times and pick the most common final answer.
    Return (best_answer, all_answers, all_raw_texts). """

    answers = []
    raw_texts = []

    for _ in range(sample_count):
        raw, _ = chain_of_thought(question_text, temp=temp)
        final = get_final_answer(raw)
        answers.append(final)
        raw_texts.append(raw)
            
    non_empty = [ a for a in answers if a ]
    if non_empty:
        counts = Counter(non_empty)
        best = counts.most_common(1)[0][0]
    else:
        best = ""

    return best, answers, raw_texts

In [119]:
# test
best, all_ans, all_raw = self_con("what is 13+29? just give number", sample_count=3)
print("Best:" , best)
print("All:", all_ans)


Best: 42
All: ['42', '42', '42']


Inference technique #3: Reflection/self_correction

In [120]:
# Inference technique #3: Reflection/self_correction
def refl(question_text: str,
         candidate_answer: str,
         temp: float =0.0):
    """ Reflection: you are a strick grader. 
    If the answer is deamed wrong, solve again and return the fixed answer."""

    #1) T or F
    judge_system = (
        "you are a strick grader. "
        "reply with True or False, no explanation"
    )

    judge_prompt =  f""" You are grading the question answer pair.
    Return what is True if CANDIDATE_ANSWER is acceptable else return False.

    QUESTION:
    {question_text}

    PREVIOUS_ANSWER:
    {candidate_answer}

    Answer only True or False. """

    judge_res = call_model_chat_completions(
        prompt=judge_prompt,
        system= judge_system,
        model=MODEL,
        temperature=temp,
    )

    judge_reply = (judge_res.get("text") or "").strip().lower()

    if judge_reply.startswith("true"):
        return candidate_answer
    
    # if wrong ask for a corrcted answer
    fix_system = (
        "You are a probelm solver who is careful."
        "Think step by step and give the final answer on the last line"
        "starting with 'FINAL ANSWER:'."
    )

    fix_prompt = f"""The previous answer was judged inccorect.

    QUESTION:
    {question_text}

    PREVIOUS_ANSWER:
    {candidate_answer}
    
    solve the question correctly, think step by step and end with: FINAL ANSWER: <correct_answer>
    """

    fix_raw, _ = chain_of_thought(fix_prompt, system_message=fix_system, temp=0.3)
    fixed = get_final_answer(fix_raw)
    return fixed or candidate_answer
    

In [121]:
#test
q = "what is 5 + 7? Just give the number."
best, all_ans, all_raw = self_con(q, sample_count=3)
print("self_con best:", best)
final =refl(q, best)
print("after reflection:", final)

self_con best: 12
after reflection: 12


Boss function to call the 3 techniques

In [122]:
# Agent which combines all 3 techniques

def agent_solve( example:dict, sc_samples: int = 2, use_reflection:bool = True) -> str:
    """ run full agen on example.
    1) self_con for CoT + self-consistency
    2) refl for reflection/ self correction
    return final answer as plain string"""

    question = example.get("input", "")

    #CoT + self-con
    best, all_ans, all_raw = self_con(
        question_text=question,
        sample_count=sc_samples,
    )

    #reflection/self-correction
    if use_reflection:
        final = refl(
            question_text=question,
            candidate_answer= best,
        )
    else:
        final = best

    return str(final).strip()

In [123]:
#test
for i in range(3):
    ex=dev_data[i]
    print("\nExample", i)
    print("INPUT", ex["input"])
    print("GOLD", ex["output"])
    print("AGENT", agent_solve(ex, sc_samples=3))



Example 0
INPUT Let $ABCD$ be a convex quadrilateral with $AB = CD = 10$ , $BC = 14$ , and $AD = 2\sqrt{65}$ . Assume that the diagonals of $ABCD$ intersect at point $P$ , and that the sum of the areas of triangles $APB$ and $CPD$ equals the sum of the areas of triangles $BPC$ and $APD$ . Find the area of quadrilateral $ABCD$ .
GOLD 112
AGENT 65

Example 1
INPUT A tennis player computes her win ratio by dividing the number of matches she has won by the total number of matches she has played. At the start of a weekend, her win ratio is exactly $0.500$ . During the weekend, she plays four matches, winning three and losing one. At the end of the weekend, her win ratio is greater than $.503$ . What's the largest number of matches she could've won before the weekend began?
GOLD 164
AGENT 1

Example 2
INPUT What is the product of the real roots of the equation $x^2 + 18x + 30 = 2 \sqrt{x^2 + 18x + 45}$ ?
GOLD 20
AGENT 2


In [124]:
#code for one chunk
import json

def run_chunk_range(start_idx: int,
                    end_idx: int,
                    sc_samples: int = 2,
                    use_reflection: bool = False,
                    chunk_id: int =0):
    """ Run agent_solve on test_data[start_idx: end_idx] and save results
    to chunk_{chunk_id}.json as a list of { "index": im "output":"..."}."""

    #laod test data fresh inside the function
    with open("cse_476_final_project_test_data.json", "r", encoding="utf-8") as f:
        test_data=json.load(f)

    n = len(test_data)
    start_idx = max(0, start_idx)
    end_idx = min(end_idx, n)

    print(f"[chunk {chunk_id}] Running indicies {start_idx}..{end_idx-1} (total {end_idx - start_idx})")
    rows = []
    for i in range( start_idx, end_idx):
        ex = test_data[i]
        ans= agent_solve(ex, sc_samples = sc_samples, use_reflection=use_reflection)
        out_str = str(ans).strip()

        rows.append({
            "index": i,
            "output": out_str
        })

        # progress log inside this chunk
        if (i - start_idx) % 50 ==0:
            print(f"[chunk {chunk_id} Done global index {i}]")

    fname = f"chunk_{chunk_id}.json"
    with open(fname, "w", encoding="utf-8") as f:
        json.dump(rows, f, indent=2)

    print(f"[chunk {chunk_id}] wrote {len(rows)} rows to {fname}")
    return fname


In [125]:
# code for 8 chunks

import math
from concurrent.futures import ThreadPoolExecutor
import json 

 # laod test data fresh inside the function
with open("cse_476_final_project_test_data.json", "r", encoding="utf-8") as f:
        test_data = json.load(f)

N = len(test_data)
print("total test questions:" , N)

# 8 chunks
num_chunks =8
chunk_size = math.ceil(N/num_chunks)
print(" chunk size:", chunk_size)

# jobs
jobs = []
for chunk_id in range(num_chunks):
        start_idx = chunk_id *chunk_size
        end_idx = min((chunk_id+1)*chunk_size, N)
        if start_idx >= N:
                break
        jobs.append((start_idx, end_idx, chunk_id))

print("jobs (start_idx, end_idx, chunk_id):")
for j in jobs:
        print(" ", j)

def job_wrapper(args):
        start_idx, end_idx, chunk_id =args
        return run_chunk_range(
                start_idx= start_idx,
                end_idx= end_idx,
                sc_samples= 2,
                use_reflection=False,
                chunk_id= chunk_id,
        )
max_workers = min(4, len(jobs))
print(f"running max workers = {max_workers}")

with ThreadPoolExecutor(max_workers=max_workers) as pool:
        results = list(pool.map(job_wrapper, jobs))

print("ALL chunks finished, Files:", results)
        



total test questions: 6208
 chunk size: 776
jobs (start_idx, end_idx, chunk_id):
  (0, 776, 0)
  (776, 1552, 1)
  (1552, 2328, 2)
  (2328, 3104, 3)
  (3104, 3880, 4)
  (3880, 4656, 5)
  (4656, 5432, 6)
  (5432, 6208, 7)
running max workers = 4
[chunk 1] Running indicies 776..1551 (total 776)
[chunk 0] Running indicies 0..775 (total 776)
[chunk 3] Running indicies 2328..3103 (total 776)
[chunk 2] Running indicies 1552..2327 (total 776)
[chunk 0 Done global index 0]
[chunk 1 Done global index 776]
[chunk 2 Done global index 1552]
[chunk 3 Done global index 2328]
[chunk 0 Done global index 50]
[chunk 1 Done global index 826][chunk 2 Done global index 1602]

[chunk 3 Done global index 2378]
[chunk 0 Done global index 100]
[chunk 1 Done global index 876]
[chunk 2 Done global index 1652]
[chunk 3 Done global index 2428]
[chunk 2 Done global index 1702][chunk 0 Done global index 150]
[chunk 1 Done global index 926]

[chunk 3 Done global index 2478]
[chunk 1 Done global index 976]
[chunk 0 Don

In [ ]:
import json

i =766
ex = test_data[i]
print("INDEX:",i)
print("DOMAIN:", ex.get("domain"))
print("INPUT:", ex.get("input"))


In [ ]:
#run agent on all test questoins and fill outputs
for i, (ex, row) in enumerate(zip(test_data, test_answers)):
    q = ex.get("input", "")
    ans = agent_solve(ex,sc_samples=3, use_reflection = False)
    row["output"] = str(ans).strip()
                        
    if i % 10 ==0:
        print(f"Done {i} question...")

# save final result file
out_path = "my_final_project_results.json"
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(test_answers, f, indent=2)

print("Wrote results to", out_path)